In [1]:
import numpy
import cPickle, gzip

In [3]:
# Set up a number of initial variables for use with baseline
NUM_TASKS = 3; # number of learning tasks (for multi-task learning)
NUM_FOLDS = 10; # number of folds for training (main cross validation loop)
NUM_EPOCH = 5; # number of epochs



truth_a_arr = []
pred_a_arr = []

truth_b_arr = []
pred_b_arr = []

truth_c_arr = []
pred_c_arr = []


In [7]:
for fold in range( NUM_FOLDS ):

    features_train = []
    labels_train = []
    truths_train = []

    features_test = []
    labels_test = []
    truths_test = []

    n_out = []

    for task in range( NUM_TASKS ):
        file_post = '.' + str(task) + '.' + str(fold) + '.pkl.gz'
        fname_train = 'train/train' + file_post; 
        fname_test  = 'test/test' + file_post; 

        with gzip.open( fname_train, 'rb' ) as f:
            feature_train, label_train = cPickle.load( f )

        with gzip.open( fname_test, 'rb') as f:
            feature_test, label_test = cPickle.load( f )

        features_train.append( feature_train )
        labels_train.append( label_train )

        features_test.append( feature_test )
        labels_test.append( label_test )

        mv = numpy.max( label_train )
        truth_train = numpy.zeros( ( len( label_train ), mv + 1 ) )
        for i in range( len( label_train ) ):
            truth_train[ i, label_train[ i ] ] = 1

        truths_train.append( truth_train )

        mv = numpy.max( label_test )
        truth_test = numpy.zeros( ( len( label_test ), mv + 1 ) )
        for i in range( len( label_test ) ):
            truth_test[ i, label_test[ i ] ] = 1

        truths_test.append( truth_test )

        n_out.append( mv + 1 )

    flen = len( feature_train[ 0 ] ); # input feature length is set to 400 for now based on the training examples available.


In [8]:
print n_out

[2, 2, 4]


In [17]:
print len(features_train[1])

1200


In [13]:
print len(truths_train[1])

1200


In [11]:
print truths_train[2]

[[ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]
